In [1]:
# cài đặt một số thư viện cần thiết
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [2]:
!pip install openmim
!mim install mmcv-full
!mim install mmengine
!pip install einops
!pip install scikit-learn
!pip install attrs
!pip install scikit-image

Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 6.0 MB/s eta 0:00:00
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached yapf-0.40.2-py3-none-any.whl.metadata (45 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 5.1 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
  Using cached mmengine-0.10.4-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00


In [3]:
# Kết nối với gg drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# clone file git
!git clone https://github.com/Westlake-AI/openmixup.git

Cloning into 'openmixup'...
remote: Enumerating objects: 9248, done.
remote: Counting objects: 100% (1435/1435), done.
remote: Compressing objects: 100% (834/834), done.
remote: Total 9248 (delta 855), reused 971 (delta 551), pack-reused 7813 (from 1)
Receiving objects: 100% (9248/9248), 3.65 MiB | 24.13 MiB/s, done.
Resolving deltas: 100% (6340/6340), done.


In [5]:
%cd openmixup


/content/openmixup


In [6]:
!python setup.py develop

running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running egg_info
creating openmixup.egg-info
writing manifest file 'openmixup.egg-info/SOURCES.txt'
writing manifest file 'openmixup.egg-info/SOURCES.txt'
running build_ext
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning:  is an invalid version and will not be supported in a future release
  warnings.warn(
Installing timm-1.0.8-py3-none-any.whl to /usr/local/lib/python3.10/dist-packages


**Cop file checkpoint từ drive sang**

In [8]:
!mkdir -p /content/openmixup/work_dirs/classification/cifar10/mixups/basic
!cp -r /content/drive/MyDrive/r18_mixups_CE_none /content/openmixup/work_dirs/classification/cifar10/mixups/basic

**Bỏ qua phần này( đây là phần train khi chưa có checkpoint)**

In [ ]:
%%writefile /content/openmixup/configs/classification/cifar10/mixups/basic/r18_mixups_CE_none.py
# - dataset: configuration of dataset:
# cifar10/sz32_bs100 ->:
# dataset: cifar10, size 32
# bs: 100
# - Model backbone: eg. https://github.com/Westlake-AI/openmixup/blob/main/configs/classification/_base_/models/resnet/resnet18_cifar.py
# type task: classification, MixupClassification, detection, segmentation,...
# backbone: type: Resnet, Resnet_cifar...
# eg: Resnet_CIFAR: https://github.com/Westlake-AI/openmixup/blob/main/openmixup/models/backbones/resnet_mmcls.py#L716
# - Type task: augmentation -> MixupClassification
# mix_mode, mix_args
# mix_mode: declare mix augmnentation algo. eg. mixup (>< smoothmix), smoothmix
# mix_args: args for correspondance mix_mode.
#

# - Runtime: leave as is.
#
#
_base_ = [
    '../../../_base_/datasets/cifar10/sz32_bs100.py',
    '../../../_base_/default_runtime.py',
]

# model settings
model = dict(
    type='MixUpClassification',
    pretrained=None,
    alpha=1,
    mix_mode="smoothmix", # Run with default mixup args.
    # mix_mode = "smoothmix",
    mix_args=dict(
        smoothmix = dict(alpha=1.0) # not used
    ),
    backbone=dict(
        # type='ResNet_CIFAR',  # CIFAR version
        type='ResNet_CIFAR',  # required by 'manifoldmix'
        # ResNet, ResNet_CIFAR ResNet_MixCIFAR
        # ref. https://github.com/Westlake-AI/openmixup/blob/main/openmixup/models/backbones/resnet_mmcls.py
        depth=18,
        num_stages=4,
        out_indices=(3,),  # no conv-1, x-1: stage-x
        style='pytorch'),
    head=dict(
        type='ClsHead',  # normal CE loss (NOT SUPPORT PuzzleMix, use soft/sigm CE instead)
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        with_avg_pool=True, multi_label=False, in_channels=512, num_classes=10)
)

# optimizer
optimizer = dict(type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)

# learning policy
lr_config = dict(policy='CosineAnnealing', min_lr=0.)

# runtime settings
runner = dict(type='EpochBasedRunner', max_epochs=2)


Overwriting /content/openmixup/configs/classification/cifar10/mixups/basic/r18_mixups_CE_none.py


**Bỏ qua phần này( đây là phần train khi chưa có checkpoint)**

In [ ]:
import torch
from torch.utils.data import DataLoader

# Huấn luyện và lưu dataset
!python -c "import torchvision; torchvision.datasets.CIFAR10(download=True, root='data/cifar10');"
!bash /content/openmixup/tools/dist_train.sh /content/openmixup/configs/classification/cifar10/mixups/basic/r18_mixups_CE_none.py 1




Files already downloaded and verified
/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/content/openmixup/openmixup/apis/train.py:22: UserWarning: Deprecat

**Chỉnh lại epoch để train tiếp, chỉnh càng lớn càng tốt**

In [12]:
%%writefile /content/openmixup/configs/classification/cifar10/mixups/basic/r18_mixups_CE_none.py
# - dataset: configuration of dataset:
# cifar10/sz32_bs100 ->:
# dataset: cifar10, size 32
# bs: 100
# - Model backbone: eg. https://github.com/Westlake-AI/openmixup/blob/main/configs/classification/_base_/models/resnet/resnet18_cifar.py
# type task: classification, MixupClassification, detection, segmentation,...
# backbone: type: Resnet, Resnet_cifar...
# eg: Resnet_CIFAR: https://github.com/Westlake-AI/openmixup/blob/main/openmixup/models/backbones/resnet_mmcls.py#L716
# - Type task: augmentation -> MixupClassification
# mix_mode, mix_args
# mix_mode: declare mix augmnentation algo. eg. mixup (>< smoothmix), smoothmix
# mix_args: args for correspondance mix_mode.
#

# - Runtime: leave as is.
#
#
_base_ = [
    '../../../_base_/datasets/cifar10/sz32_bs100.py',
    '../../../_base_/default_runtime.py',
]

# model settings
model = dict(
    type='MixUpClassification',
    pretrained=None,
    alpha=1,
    mix_mode="smoothmix", # Run with default mixup args.
    # mix_mode = "smoothmix",
    mix_args=dict(
        smoothmix = dict(alpha=1.0) # not used
    ),
    backbone=dict(
        # type='ResNet_CIFAR',  # CIFAR version
        type='ResNet_CIFAR',  # required by 'manifoldmix'
        # ResNet, ResNet_CIFAR ResNet_MixCIFAR
        # ref. https://github.com/Westlake-AI/openmixup/blob/main/openmixup/models/backbones/resnet_mmcls.py
        depth=18,
        num_stages=4,
        out_indices=(3,),  # no conv-1, x-1: stage-x
        style='pytorch'),
    head=dict(
        type='ClsHead',  # normal CE loss (NOT SUPPORT PuzzleMix, use soft/sigm CE instead)
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        with_avg_pool=True, multi_label=False, in_channels=512, num_classes=10)
)

# optimizer
optimizer = dict(type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)

# learning policy
lr_config = dict(policy='CosineAnnealing', min_lr=0.)

# runtime settings
runner = dict(type='EpochBasedRunner', max_epochs=300)


Overwriting /content/openmixup/configs/classification/cifar10/mixups/basic/r18_mixups_CE_none.py


**Lệnh train tiếp**

In [ ]:
# Huấn luyện và lưu dataset
!python -c "import torchvision; torchvision.datasets.CIFAR10(download=True, root='data/cifar10');"
!bash /content/openmixup/tools/dist_train.sh /content/openmixup/configs/classification/cifar10/mixups/basic/r18_mixups_CE_none.py 1 --resume_from /content/openmixup/work_dirs/classification/cifar10/mixups/basic/r18_mixups_CE_none/latest.pth


Files already downloaded and verified
/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/content/openmixup/openmixup/apis/train.py:22: UserWarning: Deprecat

**Lưu checkpoint**

In [ ]:
!cp -r /content/openmixup/work_dirs/classification/cifar10/mixups/basic/r18_mixups_CE_none /content/drive/MyDrive/